Import Initial Data

In [ ]:
# imports 
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
# Figure out how to import this model 
# from tensorflow.keras.applications.resnet50v2 import ResNet50V2

In [ ]:
# Note: There are only 8 images per label here, so we're going to have to change what we do here 
# Confirmed that the # of images in the 3 folders sum to 5,856, which is the total number of images 

In [ ]:
# Use validation_split to get the training and validation data 
train_data = keras.utils.image_dataset_from_directory(
    'initial_data/train',
    image_size=(180, 180), 
    batch_size=32, 
    validation_split=0.2, 
    subset = 'training', 
    seed = 10
)

In [ ]:
# Validation data: 79 files 
val_data = keras.utils.image_dataset_from_directory(
    'initial_data/train',
    image_size=(180, 180), 
    batch_size=32, 
    validation_split=0.2, 
    subset = 'validation', 
    seed = 10
)

In [ ]:
# Test data: 50 pneumonia and 50 normal images 
test_data = keras.utils.image_dataset_from_directory(
    'initial_data/test',
    image_size=(180, 180), 
    batch_size=32, 
)

Initial ResNet 50 Model:

In [ ]:
# Function to build model 
def build_resnet50_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # Resnet 50 basemodel 
    base_model = ResNet50(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
# Build the model: 
resnet50_mod = build_resnet50_model()

In [ ]:
# Function to fit model 
def fit_model(model, train_set, validation_set):
    """Fit a model with the above stated criteria"""
    # Set patience to 5 so it doesn't take too long to fit 
    early_stopping = keras.callbacks.EarlyStopping(patience = 5)
    
    model.fit(train_set, 
              validation_data = validation_set, 
              callbacks = [early_stopping], 
              epochs = 500)
    
    return model

In [ ]:
# Fit the model: 
# Started 12:18 pm; ended 12:55 pm --> 7 min to train 
# Best model: 
fitted_resnet50_mod = fit_model(resnet50_mod, train_data, val_data)

Initial Inception V3 model

In [ ]:
# Function to build model 
def build_inceptionV3_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # InceptionV3 base model 
    base_model = InceptionV3(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
# Build the model: 
inceptionV3_mod = build_inceptionV3_model()

In [ ]:
# Fit the model: 
# Started 12:48 pm; ended pm --> 9 min to train 
fitted_inceptionV3_mod = fit_model(inceptionV3_mod, train_data, val_data)